In [1]:
from platform import python_version
print(python_version())
import numpy as np
import subprocess

import pandas as pd
import dask.dataframe as dd
import hail as hl
hl.init(min_block_size=256)

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--driver-memory 700G pyspark-shell"

3.6.13


Running on Apache Spark version 2.4.1
SparkUI available at http://vega.local:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.64-1ef70187dc78
LOGGING: writing to /enadisk/maison/sage2016/weber/PycharmProjects/MISTIC_CAGI6/hail-20210928-1954-0.2.64-1ef70187dc78.log


In [26]:
# Remap assembly

def remap_subprocess_fct(input_bed, output_bed, perl_location="/usr/local/bin/perl", remap_location="/biolo/ngs/remap/remap_api.pl"):

    subprocess_args = [
        perl_location,
        remap_location,
        "--mode",
        "asm-asm",
        "--from",
        "GCF_000001405.25",
        "--dest",
        "GCF_000001405.26",
        "--annotation",
        input_bed,
        "--annot_out",
        output_bed,
        "--report_out",
        "report",
        "--in_format",
        "bed",
        "--out_format",
        "bed",
    ]
    p1 = subprocess.Popen(subprocess_args)
    p1.wait()

In [107]:
# CCRS IMPORT & REFORMAT

ccr = pd.read_csv('/gstock/biolo_datasets/variation/benchmark/Databases/CCR/ccrs_complete.bedGraph.gz', compression='gzip', sep='\t', names=['CHR', 'START', 'END', 'CCR'])
from tqdm import tqdm
for chromosome in tqdm(ccr.CHR.unique()):
    ccr.loc[ccr['CHR'] == chromosome].to_csv('/gstock/biolo_datasets/variation/benchmark/Databases/CCR/CCR_split/ccr.chr{}.bedGraph.gz'.format(str(chromosome)), compression='gzip', sep='\t', header=False, index=False)

/home/weber/.conda/envs/hail/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|██████████| 24/24 [01:22<00:00,  3.44s/it]


In [176]:
# CCR REFORMAT

ccrs_table_pd = pd.read_csv("/gstock/biolo_datasets/variation/benchmark/Databases/CCR/CCR_split/38/ccr.complete.bedGraph.gz", compression='gzip', sep='\t', names=['CHR', 'START', 'END', 'CCR'])
chroms = ['{}'.format(i) for i in list(range(1,23)) + ['X']]
ccrs_table_pd.CHR = ccrs_table_pd.CHR.astype(str)
ccrs_table_pd.loc[ccrs_table_pd['CHR'].isin(chroms), 'CHR'] = 'chr' + ccrs_table_pd.loc[ccrs_table_pd['CHR'].isin(chroms), 'CHR']
ccrs_table_pd.loc[ccrs_table_pd['CHR'].str.contains('chr')].to_csv('/gstock/biolo_datasets/variation/benchmark/Databases/CCR/CCR_split/38/ccr.complete.38.bedGraph.gz',compression='gzip', sep='\t', header=False, index=False)

In [209]:
# CCR IMPORT IN HAIL

chroms = {'{}'.format(i) : 'chr{}'.format(i) for i in list(range(1,23)) + ['X']}
ccrs_table = hl.import_bed(
    "/gstock/biolo_datasets/variation/benchmark/Databases/CCR/CCR_split/38/ccr.complete.38.bedGraph.gz",
    reference_genome="GRCh38",
    force_bgz=True,
    contig_recoding=chroms,
)


2021-09-25 23:50:07 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
  Loading field 'f3' as type str (user-supplied)


In [208]:

# MISTIC EXISTING SCORES IMPORT IN HAIL

mistic_hail = hl.import_table('/gstock/biolo_datasets/variation/benchmark/MISTIC/PRECOMPUTING_CADD/2021/MISTIC_GRCh37_website_2021.tsv.bgz', force_bgz=True)

mistic_hail = mistic_hail.annotate(
    ID  = mistic_hail['#CHROM (GRCh37 assembly)'].replace('chr', '') + '-' + mistic_hail['POS'] + '-' + mistic_hail['REF'] + '-' + mistic_hail['ALT']
)
# mistic_hail = mistic_hail.filter(mistic_hail['#CHROM (GRCh37 assembly)'] == '10')
mistic_hail = mistic_hail.key_by('ID')


2021-09-25 23:49:44 Hail: INFO: Reading table without type imputation
  Loading field '#CHROM (GRCh37 assembly)' as type str (not specified)
  Loading field 'POS' as type str (not specified)
  Loading field 'REF' as type str (not specified)
  Loading field 'ALT' as type str (not specified)
  Loading field 'MISTIC_score' as type str (not specified)
  Loading field 'MISTIC_pred' as type str (not specified)


In [212]:
# dbNSFP IMPORT IN HAIL

chroms = {'{}'.format(i) : 'chr{}'.format(i) for i in list(range(1,23)) + ['X', 'Y', 'M']}
# chroms = {'{}'.format(i) : 'chr{}'.format(i) for i in [10]}
dbnsfp_vcf_hail = hl.import_vcf('/gstock/MISTIC/CAGI6/dbnsfp_complete.vep.vcf.gz', reference_genome="GRCh38", force_bgz=True, contig_recoding=chroms)
# dbnsfp_test['id'] = dbnsfp_test["#chr"].astype(str) + '-' + dbnsfp_test["pos(1-coor)"].astype(str) + '-' + dbnsfp_test["ref"].astype(str) + '-' + dbnsfp_test["alt"].astype(str) + '-' + dbnsfp_test["hg19_chr"].astype(str) + '-' + dbnsfp_test["hg19_pos(1-based)"].astype(str) 

dbnsfp_vcf_hail = dbnsfp_vcf_hail.annotate_rows(
    CSQ_split = dbnsfp_vcf_hail.info.CSQ.map(lambda x: x.split('\|'))
)

dbnsfp_vcf_hail = dbnsfp_vcf_hail.annotate_rows(
    Allele = dbnsfp_vcf_hail.CSQ_split.map(lambda x: x[0]),
    Consequence = dbnsfp_vcf_hail.CSQ_split.map(lambda x: x[1]),
    IMPACT = dbnsfp_vcf_hail.CSQ_split.map(lambda x: x[2]),
    SYMBOL = dbnsfp_vcf_hail.CSQ_split.map(lambda x: x[3]),
    Gene = dbnsfp_vcf_hail.CSQ_split.map(lambda x: x[4]),
    CONDEL = dbnsfp_vcf_hail.CSQ_split.map(lambda x: x[36]),
)

dbnsfp_vcf_hail = dbnsfp_vcf_hail.annotate_rows(
    MISSENSE = dbnsfp_vcf_hail.Consequence.contains('missense_variant'),
    STOPGAINED = dbnsfp_vcf_hail.Consequence.contains('stop_gained'),
    STOPLOST = dbnsfp_vcf_hail.Consequence.contains('stop_lost'),
    STARTLOST = dbnsfp_vcf_hail.Consequence.contains('start_lost'),
#     INFO = dbnsfp_vcf_hail.info.INFO[0],
    ID_raw = dbnsfp_vcf_hail.info.INFO[0].split('-'),
)

dbnsfp_vcf_hail = dbnsfp_vcf_hail.annotate_rows(
    ID = dbnsfp_vcf_hail.ID_raw[4] + '-' + dbnsfp_vcf_hail.ID_raw[5] + '-' + dbnsfp_vcf_hail.ID_raw[2] + '-' + dbnsfp_vcf_hail.ID_raw[3]
    
)

dbnsfp_vcf_hail = dbnsfp_vcf_hail.annotate_rows(
    MISTIC = hl.float(mistic_hail[dbnsfp_vcf_hail.ID].MISTIC_score),
    CCR = hl.float(ccrs_table[dbnsfp_vcf_hail.locus].target),
)

In [ ]:
# CONVERT WITH HAIL VCF TO CSV 

dbnsfp_vcf_hail.rows().export('/gstock/MISTIC/CAGI6/dbnsfp_complete.vep.csv.bgz')


2021-09-25 23:52:30 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-25 23:53:05 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-25 23:55:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-25 23:56:20 Hail: INFO: Ordering unsorted dataset with network shuffle


In [ ]:
# IMPORT INTO PANDAS

dbnsfp_vcf_hail_pandas = pd.read_csv('/gstock/MISTIC/CAGI6/dbnsfp_complete.vep.csv.bgz', compression='gzip', sep='\t')
dbnsfp_vcf_hail_pandas

In [ ]:
# CONVERT TO PARQUET FOR MORE EFFICIENT R/W

dbnsfp_vcf_hail_pandas.to_parquet('/gstock/MISTIC/CAGI6/dbnsfp_complete.vep.parquet', index=False)

# dbNSFP PANDAS

In [2]:
dbnsfp_vcf_hail_pandas = pd.read_parquet('/gstock/MISTIC/CAGI6/dbnsfp_complete.vep.parquet')
dbnsfp_vcf_hail_pandas

,locus,alleles,rsid,qual,filters,info,CSQ_split,Allele,Consequence,IMPACT,...,Gene,CONDEL,MISSENSE,STOPGAINED,STOPLOST,STARTLOST,ID_raw,ID,MISTIC,CCR
0,chr1:65565,"[""A"",""C""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""C"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""C"",""C""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,"[""ENSG00000240361"",""ENSG00000186092""]","["""",""neutral(0.000)""]",False,False,False,True,"[""1"",""65565"",""A"",""C"",""1"",""65565""]",1-65565-A-C,NaN,NaN
1,chr1:65565,"[""A"",""G""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""G"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""G"",""G""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,"[""ENSG00000240361"",""ENSG00000186092""]","["""",""neutral(0.349)""]",False,False,False,True,"[""1"",""65565"",""A"",""G"",""1"",""65565""]",1-65565-A-G,NaN,NaN
2,chr1:65565,"[""A"",""T""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""T"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""T"",""T""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,"[""ENSG00000240361"",""ENSG00000186092""]","["""",""neutral(0.000)""]",False,False,False,True,"[""1"",""65565"",""A"",""T"",""1"",""65565""]",1-65565-A-T,NaN,NaN
3,chr1:65566,"[""T"",""A""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""A"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""A"",""A""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,"[""ENSG00000240361"",""ENSG00000186092""]","["""",""neutral(0.445)""]",False,False,False,True,"[""1"",""65566"",""T"",""A"",""1"",""65566""]",1-65566-T-A,NaN,NaN
4,chr1:65566,"[""T"",""C""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""C"",""start_lost"",""HIGH"",""OR4F5"",""ENSG0000018...","[""C"",""C""]","[""start_lost"",""downstream_gene_variant""]","[""HIGH"",""MODIFIER""]",...,"[""ENSG00000186092"",""ENSG00000240361""]","[""neutral(0.406)"",""""]",False,False,False,True,"[""1"",""65566"",""T"",""C"",""1"",""65566""]",1-65566-T-C,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81782918,chrM:14672,"[""A"",""G""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""G"",""intergenic_variant"",""MODIFIER"","""","""",""""...","[""G""]","[""intergenic_variant""]","[""MODIFIER""]",...,"[""""]","[""""]",False,False,False,False,"[""M"",""14672"",""A"",""G"",""M"",""14673""]",M-14673-A-G,NaN,NaN
81782919,chrM:14672,"[""A"",""T""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""T"",""intergenic_variant"",""MODIFIER"","""","""",""""...","[""T""]","[""intergenic_variant""]","[""MODIFIER""]",...,"[""""]","[""""]",False,False,False,False,"[""M"",""14672"",""A"",""T"",""M"",""14673""]",M-14673-A-T,NaN,NaN
81782920,chrM:14673,"[""T"",""A""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""A"",""intergenic_variant"",""MODIFIER"","""","""",""""...","[""A""]","[""intergenic_variant""]","[""MODIFIER""]",...,"[""""]","[""""]",False,False,False,False,"[""M"",""14673"",""T"",""A"",""M"",""14674""]",M-14674-T-A,NaN,NaN
81782921,chrM:14673,"[""T"",""C""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""C"",""intergenic_variant"",""MODIFIER"","""","""",""""...","[""C""]","[""intergenic_variant""]","[""MODIFIER""]",...,"[""""]","[""""]",False,False,False,False,"[""M"",""14673"",""T"",""C"",""M"",""14674""]",M-14674-T-C,NaN,NaN


In [3]:

# REFORMAT dbNSFP with pandas

# dbnsfp_vcf_hail_dask = dd.read_parquet('/gstock/MISTIC/CAGI6/dbnsfp_complete.vep.parquet')
dbnsfp_vcf_hail_pandas['CHROM'] = dbnsfp_vcf_hail_pandas['locus'].apply(lambda r: r.split(':')[0].replace('chr', ''),)
dbnsfp_vcf_hail_pandas['CHROM_37'] = dbnsfp_vcf_hail_pandas['ID'].apply(lambda r: r.split('-')[0], )
dbnsfp_vcf_hail_pandas['FULL_ID'] = dbnsfp_vcf_hail_pandas['ID_raw'].apply(lambda r: "-".join(eval(r)), )

# Select missense
dbnsfp_vcf_hail_pandas['Missense_index'] = dbnsfp_vcf_hail_pandas['Consequence'].apply(lambda r: [j for j,e in enumerate(eval(r)) if 'missense' in e])
dbnsfp_vcf_hail_pandas.loc[dbnsfp_vcf_hail_pandas['Consequence'].str.contains('missense'), 'MISSENSE'] = True
# dbnsfp_vcf_hail_dask

In [15]:
dbnsfp_vcf_hail_pandas.head(25)

,locus,alleles,rsid,qual,filters,info,CSQ_split,Allele,Consequence,IMPACT,...,STOPLOST,STARTLOST,ID_raw,ID,MISTIC,CCR,CHROM,CHROM_37,FULL_ID,Missense_index
0,chr1:65565,"[""A"",""C""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""C"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""C"",""C""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,False,True,"[""1"",""65565"",""A"",""C"",""1"",""65565""]",1-65565-A-C,NaN,NaN,1,1,1-65565-A-C-1-65565,[]
1,chr1:65565,"[""A"",""G""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""G"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""G"",""G""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,False,True,"[""1"",""65565"",""A"",""G"",""1"",""65565""]",1-65565-A-G,NaN,NaN,1,1,1-65565-A-G-1-65565,[]
2,chr1:65565,"[""A"",""T""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""T"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""T"",""T""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,False,True,"[""1"",""65565"",""A"",""T"",""1"",""65565""]",1-65565-A-T,NaN,NaN,1,1,1-65565-A-T-1-65565,[]
3,chr1:65566,"[""T"",""A""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""A"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""A"",""A""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,False,True,"[""1"",""65566"",""T"",""A"",""1"",""65566""]",1-65566-T-A,NaN,NaN,1,1,1-65566-T-A-1-65566,[]
4,chr1:65566,"[""T"",""C""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""C"",""start_lost"",""HIGH"",""OR4F5"",""ENSG0000018...","[""C"",""C""]","[""start_lost"",""downstream_gene_variant""]","[""HIGH"",""MODIFIER""]",...,False,True,"[""1"",""65566"",""T"",""C"",""1"",""65566""]",1-65566-T-C,NaN,NaN,1,1,1-65566-T-C-1-65566,[]
5,chr1:65566,"[""T"",""G""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""G"",""start_lost"",""HIGH"",""OR4F5"",""ENSG0000018...","[""G"",""G""]","[""start_lost"",""downstream_gene_variant""]","[""HIGH"",""MODIFIER""]",...,False,True,"[""1"",""65566"",""T"",""G"",""1"",""65566""]",1-65566-T-G,NaN,NaN,1,1,1-65566-T-G-1-65566,[]
6,chr1:65567,"[""G"",""A""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""A"",""start_lost"",""HIGH"",""OR4F5"",""ENSG0000018...","[""A"",""A""]","[""start_lost"",""downstream_gene_variant""]","[""HIGH"",""MODIFIER""]",...,False,True,"[""1"",""65567"",""G"",""A"",""1"",""65567""]",1-65567-G-A,NaN,NaN,1,1,1-65567-G-A-1-65567,[]
7,chr1:65567,"[""G"",""C""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""C"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""C"",""C""]","[""downstream_gene_variant"",""start_lost""]","[""MODIFIER"",""HIGH""]",...,False,True,"[""1"",""65567"",""G"",""C"",""1"",""65567""]",1-65567-G-C,NaN,NaN,1,1,1-65567-G-C-1-65567,[]
8,chr1:65567,"[""G"",""T""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""T"",""start_lost"",""HIGH"",""OR4F5"",""ENSG0000018...","[""T"",""T""]","[""start_lost"",""downstream_gene_variant""]","[""HIGH"",""MODIFIER""]",...,False,True,"[""1"",""65567"",""G"",""T"",""1"",""65567""]",1-65567-G-T,NaN,NaN,1,1,1-65567-G-T-1-65567,[]
9,chr1:65568,"[""A"",""C""]",NaN,-10.0,NaN,"{""NS"":null,""DP"":null,""AF"":null,""AA"":null,""DB"":...","[[""C"",""downstream_gene_variant"",""MODIFIER"",""OR...","[""C"",""C""]","[""downstream_gene_variant"",""missense_variant""]","[""MODIFIER"",""MODERATE""]",...,False,False,"[""1"",""65568"",""A"",""C"",""1"",""65568""]",1-65568-A-C,NaN,NaN,1,1,1-65568-A-C-1-65568,[1]


In [16]:
# Split into chunks for each chr

for chromosome in dbnsfp_vcf_hail_pandas.CHROM.unique():
    print(chromosome)
    dbnsfp_vcf_hail_pandas.loc[dbnsfp_vcf_hail_pandas['CHROM'] == chromosome].to_parquet('/gstock/MISTIC/CAGI6/dbnsfp_vep_mistic_chr{}.parquet'.format(str(chromosome)), index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
X
Y
M


In [33]:
test = dbnsfp_vcf_hail_pandas.head(500)[['locus','alleles','MISTIC', 'IMPACT', 'Consequence']]

test['#chr'] = test['locus'].apply(lambda r: r.split(':')[0].replace('chr',''))
test['#chr'] = test['#chr'].astype(str)
test['pos(1-based)'] = test['locus'].apply(lambda r: r.split(':')[1])
test['ref'] = test['alleles'].apply(lambda r: eval(r)[0])
test['alt'] = test['alleles'].apply(lambda r: eval(r)[1])
test = test.rename({'MISTIC' : 'score'}, axis=1)
test.loc[(test['IMPACT'].str.contains('HIGH')) & (test['score'].isna() == True), 'comments'] = 'No missense in VEP predictions / HIGH impact'
test.loc[(test['IMPACT'].str.contains('LOW|MODIFIER')) & (test['score'].isna() == True) & (test['comments'].isna() == True), 'comments'] = 'No missense in VEP predictions / LOW|MODIFIER impact'


test.loc[test['score'] >= 0.5, 'pred'] = 'D'
test.loc[test['score'] < 0.5, 'pred'] = 'T'
test.loc[test['score'].isna() == True, 'pred'] = 'U'


test.loc[(test['IMPACT'].str.contains('HIGH')) & (test['score'].isna() == True) , 'score'] = 1
test.loc[(test['IMPACT'].str.contains('LOW|MODIFIER')) & (test['score'].isna() == True) , 'score'] = 0


test['sd'] = 0
test['comments'] = test['comments'].fillna('*')
test
test[['#chr', 'pos(1-based)','ref','alt','score','sd','pred', 'comments']].to_csv('/gstock/MISTIC/CAGI6/download/TEST/test.csv', sep='\t', index=False)
test[['#chr', 'pos(1-based)','ref','alt','score','sd','pred', 'comments', 'IMPACT', 'Consequence']]

,#chr,pos(1-based),ref,alt,score,sd,pred,comments,IMPACT,Consequence
0,1,65565,A,C,1.00000,0,U,No missense in VEP predictions / HIGH impact,"[""MODIFIER"",""HIGH""]","[""downstream_gene_variant"",""start_lost""]"
1,1,65565,A,G,1.00000,0,U,No missense in VEP predictions / HIGH impact,"[""MODIFIER"",""HIGH""]","[""downstream_gene_variant"",""start_lost""]"
2,1,65565,A,T,1.00000,0,U,No missense in VEP predictions / HIGH impact,"[""MODIFIER"",""HIGH""]","[""downstream_gene_variant"",""start_lost""]"
3,1,65566,T,A,1.00000,0,U,No missense in VEP predictions / HIGH impact,"[""MODIFIER"",""HIGH""]","[""downstream_gene_variant"",""start_lost""]"
4,1,65566,T,C,1.00000,0,U,No missense in VEP predictions / HIGH impact,"[""HIGH"",""MODIFIER""]","[""start_lost"",""downstream_gene_variant""]"
...,...,...,...,...,...,...,...,...,...,...
495,1,69239,C,T,0.31938,0,T,*,"[""MODERATE""]","[""missense_variant""]"
496,1,69241,C,A,0.51261,0,D,*,"[""MODERATE""]","[""missense_variant""]"
497,1,69241,C,G,0.23673,0,T,*,"[""MODERATE""]","[""missense_variant""]"
498,1,69241,C,T,0.20214,0,T,*,"[""MODERATE""]","[""missense_variant""]"


In [88]:
gnomad_cols = [e for e in feature_cols if 'gnomAD' in e ]
gnomad_cols

['gnomAD_exomes_AF',
 'gnomAD_exomes_AFR_AF',
 'gnomAD_exomes_AMR_AF',
 'gnomAD_exomes_EAS_AF',
 'gnomAD_exomes_NFE_AF',
 'gnomAD_exomes_SAS_AF']

In [173]:
mistic = pd.read_csv('/gstock/MISTIC/CAGI6/download/OUTPUT/MISTIC_chr{}.tsv'.format('21'), sep='\t')
pd.merge(test, mistic.rename({c:"MISTIC_"+c for c in list(mistic.columns[4:])}, axis=1), on=['#chr', 'pos(1-based)', 'ref', 'alt'])

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,CLNSIG,Gene,...,clinvar_OMIM_id,clinvar_Orphanet_id,Interpro_domain,GTEx_V8_gene,GTEx_V8_tissue,Geuvadis_eQTL_target_gene,MISTIC_score,MISTIC_sd,MISTIC_pred,MISTIC_comments
0,21,27012134,21_27012134_A_G,A,G,.,.,"ALLELEID=818128;CLNDISDB=MONDO:MONDO:0032938,M...",Pathogenic,JAM2,...,618824,.,.;.;.,.,.,.,0.242621,0,T,M/V
1,21,27066149,21_27066149_G_A,G,A,.,.,"ALLELEID=818131;CLNDISDB=MONDO:MONDO:0032938,M...",Pathogenic,JAM2,...,618824,.,.;Immunoglobulin V-set domain|Immunoglobulin-l...,.,.,.,0.641090,0,D,R/H
2,21,27071098,21_27071098_G_C,G,C,.,.,"ALLELEID=818129;CLNDISDB=MONDO:MONDO:0032938,M...",Pathogenic,JAM2,...,618824,.,.;Immunoglobulin-like domain|Immunoglobulin su...,.,.,.,0.933320,0,D,W/C
3,21,27264120,21_27264120_C_T,C,T,.,.,"ALLELEID=626283;CLNDISDB=MONDO:MONDO:0005620,M...",Likely_pathogenic,APP,...,605714,ORPHA85458,"Amyloidogenic glycoprotein, amyloid-beta pepti...",.,.,.,0.799610,0,D,G/S
4,21,28213391,21_28213391_G_T,G,T,.,.,"ALLELEID=918095;CLNDISDB=MONDO:MONDO:0005387,M...",Likely_pathogenic,ADAMTS1,...,PS311360,.,"Peptidase M12B, ADAM/reprolysin|Peptidase M12B...",.,.,.,0.152990,0,T,S/Y
5,21,33038797,21_33038797_T_C,T,C,.,.,"ALLELEID=861431;CLNDISDB=MONDO:MONDO:0007103,M...",Likely_pathogenic,SOD1,...,105400,.,"Superoxide dismutase, copper/zinc binding doma...",.,.,.,0.888600,0,D,S/P
6,21,33039674,21_33039674_G_A,G,A,.,.,ALLELEID=920983;CLNDISDB=MedGen:CN517202;CLNDN...,Likely_pathogenic,SOD1,...,.,.,"Superoxide dismutase, copper/zinc binding doma...",.,.,.,0.913210,0,D,G/S
7,21,33040802,21_33040802_G_A,G,A,.,.,"ALLELEID=861434;CLNDISDB=MONDO:MONDO:0007103,M...",Likely_pathogenic,SOD1,...,105400,.,"Superoxide dismutase, copper/zinc binding doma...",.,.,.,0.928640,0,D,D/N
8,21,33040803,21_33040803_A_C,A,C,.,.,"ALLELEID=961894;CLNDISDB=MONDO:MONDO:0007103,M...",Likely_pathogenic,SOD1,...,105400,.,"Superoxide dismutase, copper/zinc binding doma...",.,.,.,0.939960,0,D,D/A
9,21,33040822,21_33040822_T_G,T,G,.,.,"ALLELEID=1052613;CLNDISDB=MONDO:MONDO:0007103,...",Likely_pathogenic,SOD1,...,.,.,"Superoxide dismutase, copper/zinc binding doma...",.,.,.,0.738720,0,D,N/K


In [167]:
# test = pd.merge(clinvar_2021_clean, dbnsfp_tmp, on ='ID')
test.loc[test['ID'] == '21_44483148_G_T'][test.columns[:25]]


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,CLNSIG,Gene,...,aaalt,rs_dbSNP,hg19_chr,hg19_pos(1-based),hg18_chr,hg18_pos(1-based),aapos,genename,Ensembl_geneid,Ensembl_transcriptid
54,21,44483148,21_44483148_G_T,G,T,.,.,ALLELEID=1174877;CLNDISDB=MedGen:CN517202;CLND...,Likely_pathogenic,CBS,...,Q,.,21,44483148,21,43356217,290;290;290;290;290,CBSL;CBSL;CBSL;CBSL;CBSL,ENSG00000274276;ENSG00000274276;ENSG0000027427...,ENST00000624406;ENST00000398168;ENST0000061802...
55,21,44483148,21_44483148_G_T,G,T,.,.,ALLELEID=1174877;CLNDISDB=MedGen:CN517202;CLND...,Likely_pathogenic,CBS,...,Q,rs760912339,21,44483148,21,43356217,290;290;290;290,CBS;CBS;CBS;CBS,ENSG00000160200;ENSG00000160200;ENSG0000016020...,ENST00000398158;ENST00000398165;ENST0000035962...


In [ ]:

# EXPORT FILES 

from tqdm import tqdm
tqdm.pandas()
import time 

import os, sys
sys.path.append('/gstock/biolo_datasets/variation/benchmark/MISTIC/RADMEL_git/Complementary_tools')
from AAIndex import AAIndex
a = AAIndex()

# Fct to retrieve AAIndex properities in AAIndex class
def aa_index_pandas(r):
    try : 
        s = pd.Series(a.score_missense(r['aaref'],r['aaalt']))
        r = pd.concat([r, s])
        return r
    except :
        print(r['aaref'],r['aaalt'])
        return r

log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'w')
log.close()

chrs = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y', 'M']
for chromosome in chrs:
    output_file_path = '/gstock/MISTIC/CAGI6/download/OUTPUT/dbnsfp.final.chr{}.csv'.format(chromosome)
    print("{}{}{}".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), " - CHROM : ", chromosome))
    log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
    log.write("{}{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), " - CHROM : ", chromosome))
    log.close()
    
    if os.path.isfile(output_file_path) is False:
        
        # RETRIEVE DBNSFP LITE PANDAS PART CORRESPONDING TO CHR
        dbnsfp_vcf_hail_pandas_tmp = dbnsfp_vcf_hail_pandas.loc[dbnsfp_vcf_hail_pandas['CHROM'] == chromosome]
        
        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), ' - VEP VCF - ', dbnsfp_vcf_hail_pandas_tmp.shape))
        log.close()

        # RETRIEVE DBNSFP PART WITH ALL ANNOTATIONS
        dbnsfp_annotation_dask = pd.read_csv('/gstock/biolo_datasets/variation/benchmark/Annot_datasets/dbNSFP/v4.2/dbNSFP4.2a_variant.chr{}.gz'.format(chromosome), compression='gzip', sep='\t', low_memory=False)
        dbnsfp_annotation_dask['FULL_ID'] = dbnsfp_annotation_dask['#chr'].astype(str) + '-' + dbnsfp_annotation_dask['pos(1-based)'].astype(str)  + '-' + dbnsfp_annotation_dask['ref'].astype(str) + '-' + dbnsfp_annotation_dask['alt'].astype(str) + '-' + dbnsfp_annotation_dask['hg19_chr'].astype(str)  + '-' + dbnsfp_annotation_dask['hg19_pos(1-based)'].astype(str)


        # RETRIEVE FEATURE COLS
        base_cols = ['FULL_ID', '#chr', 'pos(1-based)', 'ref', 'alt', 'aaref', 'aaalt', 'rs_dbSNP', 'hg19_chr', 'hg19_pos(1-based)', 'hg18_chr', 'hg18_pos(1-based)', 'aapos', 'genename', 'Ensembl_geneid', 'Ensembl_transcriptid', 'Ensembl_proteinid']
        feature_cols = sorted(["gnomAD_exomes_AF", "MetaSVM_score", "MetaLR_score", "VEST4_score", "gnomAD_exomes_ASJ_AF", "gnomAD_exomes_FIN_AF", "gnomAD_exomes_NFE_AF", "MPC_score", "CADD_phred", "SIFT_score", "phyloP100way_vertebrate", "29way_logOdds", "Polyphen2_HVAR_score", "GERP++_RS", "phastCons17way_primate", "phastCons30way_mammalian", "phyloP30way_mammalian", "gnomAD_exomes_SAS_AF", "phastCons100way_vertebrate", "phyloP17way_primate", "gnomAD_exomes_AMR_AF", "gnomAD_exomes_EAS_AF", "gnomAD_exomes_AFR_AF", ])
        gnomad_cols = [e for e in feature_cols if 'gnomAD' in e ]

        dbnsfp_annotation_dask = dbnsfp_annotation_dask.rename({'SiPhy_29way_logOdds' : '29way_logOdds'}, axis=1)
        dbnsfp_annotation_lite = dbnsfp_annotation_dask[base_cols + feature_cols]
        dbnsfp_annotation_lite = dbnsfp_annotation_lite.replace('.', np.nan)
        dbnsfp_annotation_lite[gnomad_cols] = dbnsfp_annotation_lite[gnomad_cols].fillna(0)
        dbnsfp_annotation_lite = dbnsfp_annotation_lite[base_cols + feature_cols]
        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), '- dbNSFP lite  ', dbnsfp_annotation_lite.shape))
        log.close()


        # MERGE ON VARIANT ID
        dbnsfp_annotation_complete_dask = pd.merge(dbnsfp_vcf_hail_pandas_tmp, dbnsfp_annotation_lite,on=['FULL_ID'], how='left')
        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), '- VEP VCF X dbNSFP lite - ', dbnsfp_annotation_complete_dask.shape))
        log.close()

        base_cols = base_cols + ['MISTIC', 'IMPACT', 'Consequence']
#         print(dbnsfp_annotation_complete_dask.columns.tolist())
#         print(dbnsfp_annotation_complete_dask[['FULL_ID', 'IMPACT', 'Consequence', 'MISTIC']])
        
        # RETRIEVE VARIANTS WITHOUT MISTIC SCORE
        dbnsfp_annotation_complete_dask_missense_mistic = dbnsfp_annotation_complete_dask.loc[(dbnsfp_annotation_complete_dask['MISTIC'].isna() == True)]
        dbnsfp_annotation_complete_dask_others = dbnsfp_annotation_complete_dask.loc[~dbnsfp_annotation_complete_dask.index.isin(dbnsfp_annotation_complete_dask_missense_mistic.index.tolist())]
        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), ' - missense WITHOUT MISTIC - ',dbnsfp_annotation_complete_dask_missense_mistic.shape))
        log.write("{}{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), ' - OTHERS - ',dbnsfp_annotation_complete_dask_others.shape))
        log.close()


        # HANDLE CONDEL SPECIFIC CASE
        dbnsfp_annotation_complete_dask_missense_mistic['CONDEL'] = dbnsfp_annotation_complete_dask_missense_mistic['CONDEL'].fillna("['neutral(0.5)']").apply(lambda r: np.mean([float(e.split('(')[1].split(')')[0]) for e in eval(r) if e != ""]))
        dbnsfp_annotation_complete_dask_missense_mistic = dbnsfp_annotation_complete_dask_missense_mistic.rename({'CONDEL' : 'Condel'}, axis=1)
        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), '- Selecting values ... - '))
        log.close()
        
        # COMPUTE MEDIAN FOR ROWS WITH MULTIPLE VALUES
        def select_values(r):
             return r.apply(lambda e: np.median([float(sub_e) for sub_e in str(e).split(';') if sub_e != '.']))
        dbnsfp_annotation_complete_dask_missense_mistic[feature_cols] = dbnsfp_annotation_complete_dask_missense_mistic[feature_cols].apply(lambda r: select_values(r), axis=1)

        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), '- Selecting values finished ! - '))
        log.close()
        
        feature_cols = sorted(feature_cols + ['CCR', 'Condel'])

        # MEDIAN DICT IF ANNOTATION NOT AVAILABLE
        ## WAS PRECOMPUTED ON dbNSFP DURING MISTIC DVT
        median_dict = {
                "29way_logOdds": 14,
                "CADD_phred": 20,
                "CONDEL": 0.5,
                "CCR" : 0,
                "GERP++_RS": 4.8,
                "Grantham": 70,
                "HI_score": 0.25,
                "MetaLR_score": 0.5,
                "MetaSVM_score": 0.82257,
                "MPC_score" : 0.5264,
                "phastCons100way_vertebrate": 1,
                "phastCons17way_primate": 0.8,
                "phastCons30way_mammalian": 0.9,
                "phyloP100way_vertebrate": 5.5,
                "phyloP17way_primate": 0.6,
                "phyloP30way_mammalian": 1.1,
                "Polyphen2_HVAR_score": 0.8,
                "SIFT_score": 0.05,
                "VEST4_score": 0.5,
        }
        # FILLNA WITH MEDIANS
        dbnsfp_annotation_complete_dask_missense_mistic[feature_cols] = dbnsfp_annotation_complete_dask_missense_mistic[feature_cols].fillna(median_dict)


        # DROP DUPLICATES ON AA
        combi_aa_missense = dbnsfp_annotation_dask[['aaref', 'aaalt']].drop_duplicates()
        # RETRIEVE AAINDEX SCORES
        combi_aa_missense = combi_aa_missense.reset_index(drop=True).apply(lambda r : aa_index_pandas(r), axis=1)
        aaindex_to_keep = ["HUTJ700101", "KOSJ950114", "LINK010101", "OVEJ920104", "KOSJ950109", "KOSJ950112", "ZHAC000102", "OVEJ920105", "KOSJ950110", "KOSJ950113", "KOSJ950106", "KOSJ950105", "ZHAC000105", "KOSJ950115", "KOSJ950101", "KOSJ950108", "KOSJ950111", "KOSJ950104", "DOSZ010102", "QU_C930101", "TOBD000102", "MOOG990101", "KESO980102", "BRYS930101", "SIMK990104", "NIEK910102", "MIYT790101", "NIEK910101", "VENM980101", "JOHM930101", "MIYS930101", "SIMK990103", "BENS940104", "QU_C930102", "THOP960101", "LUTR910106", "OGAK980101", "KANM000101", "BONM030104", "QUIB020101", "TANS760101", "RISJ880101", "MIYS960101", "ZHAC000106", "MIRL960101", "KAPO950101", "PRLA000102", "CSEM940101", "MIYS850102", "BENS940102", "MIYS850103", "SKOJ000101", "MIYS960102", "LUTR910101", "MIYS990107", "LUTR910107", "LUTR910104", "MUET010101", "LUTR910102", "SKOJ970101", "RUSR970101", "BONM030102", "NGPC000101", "BENS940103", "AZAE970101", "VOGG950101", "AZAE970102", "DAYM780302", "JOND940101", "GONG920101", "GEOD900101", "HENS920102", "RUSR970102", "MOHR870101", "NAOD960101", "QU_C930103", "MUET020101", "ALTS910101", "DAYM780301", "OVEJ920101", "JOND920103", "WEIL970102", "HENS920104", "WEIL970101", "CROG050101", "FEND850101", "HENS920101", "MUET020102", "MCLA720101", "FITW660101", ]
        combi_aa_missense = combi_aa_missense[['aaref', 'aaalt'] + aaindex_to_keep]

        # MERGE DATAFRAMES
        dbnsfp_annotation_complete_dask_missense_mistic = pd.merge(dbnsfp_annotation_complete_dask_missense_mistic, combi_aa_missense, on=['aaref', 'aaalt'])

        feature_cols = sorted(feature_cols + aaindex_to_keep)
        dbnsfp_annotation_complete_dask_missense_mistic[feature_cols] = dbnsfp_annotation_complete_dask_missense_mistic[feature_cols].fillna(dbnsfp_annotation_complete_dask_missense_mistic[feature_cols].median())


        dbnsfp_annotation_complete_dask_missense_mistic = dbnsfp_annotation_complete_dask_missense_mistic[base_cols + feature_cols]

        
        # PREDICT MISTIC MISSING SCORES
        dbnsfp_annotation_complete_dask_missense_mistic['MISTIC_VC'] = mistic_vc.predict_proba(dbnsfp_annotation_complete_dask_missense_mistic[feature_cols].values)[:,1]
        dbnsfp_annotation_complete_dask_missense_mistic['MISTIC_LR'] = mistic_lr.predict_proba(dbnsfp_annotation_complete_dask_missense_mistic[feature_cols].values)[:,1]
        dbnsfp_annotation_complete_dask_missense_mistic.loc[dbnsfp_annotation_complete_dask_missense_mistic['gnomAD_exomes_AF'] == 0, 'MISTIC'] = dbnsfp_annotation_complete_dask_missense_mistic.loc[dbnsfp_annotation_complete_dask_missense_mistic['gnomAD_exomes_AF'] == 0]['MISTIC_LR']
        dbnsfp_annotation_complete_dask_missense_mistic.loc[dbnsfp_annotation_complete_dask_missense_mistic['gnomAD_exomes_AF'] != 0, 'MISTIC'] = dbnsfp_annotation_complete_dask_missense_mistic.loc[dbnsfp_annotation_complete_dask_missense_mistic['gnomAD_exomes_AF'] != 0]['MISTIC_VC']

        # CONCAT MISTIC & OTHERS
        dbnsfp_annotation_complete_chr_final = pd.concat([dbnsfp_annotation_complete_dask_missense_mistic, dbnsfp_annotation_complete_dask_others])[['FULL_ID', 'MISTIC', 'IMPACT', 'Consequence', 'aaref', 'aaalt']]
        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), " - CONCAT FINAL : ", dbnsfp_annotation_complete_chr_final.shape))
        log.close()
        
        # FINAL FILE FOR OUTPUT
        dbnsfp_annotation_complete_chr_final['#chr'] = dbnsfp_annotation_complete_chr_final['FULL_ID'].apply(lambda r: r.split('-')[0].replace('chr',''))
        dbnsfp_annotation_complete_chr_final['#chr'] = dbnsfp_annotation_complete_chr_final['#chr'].astype(str)
        dbnsfp_annotation_complete_chr_final['pos(1-based)'] = dbnsfp_annotation_complete_chr_final['FULL_ID'].apply(lambda r: int(r.split('-')[1]))
        dbnsfp_annotation_complete_chr_final['ref'] = dbnsfp_annotation_complete_chr_final['FULL_ID'].apply(lambda r: r.split('-')[2])
        dbnsfp_annotation_complete_chr_final['alt'] = dbnsfp_annotation_complete_chr_final['FULL_ID'].apply(lambda r: r.split('-')[3])
        dbnsfp_annotation_complete_chr_final = dbnsfp_annotation_complete_chr_final.rename({'MISTIC' : 'score'}, axis=1)
        dbnsfp_annotation_complete_chr_final = dbnsfp_annotation_complete_chr_final.dropna(subset=['aaref', 'aaalt'])
        dbnsfp_annotation_complete_chr_final['comments'] = dbnsfp_annotation_complete_chr_final['aaref'] + '/' + dbnsfp_annotation_complete_chr_final['aaalt']
        dbnsfp_annotation_complete_chr_final
        dbnsfp_annotation_complete_chr_final.loc[dbnsfp_annotation_complete_chr_final['comments'].str.contains('X'), 'comments'] = dbnsfp_annotation_complete_chr_final.loc[dbnsfp_annotation_complete_chr_final['comments'].str.contains('X'), 'comments'] + " - STOP GAINED/LOST / Hypothetic score"
        dbnsfp_annotation_complete_chr_final_bak = dbnsfp_annotation_complete_chr_final.copy()
#         print(dbnsfp_annotation_complete_chr_final_bak)
        
#         dbnsfp_annotation_complete_chr_final.loc[(dbnsfp_annotation_complete_chr_final['IMPACT'].str.contains('HIGH')) & (dbnsfp_annotation_complete_chr_final['score'].isna() == True), 'Suppl_comments'] = 'No missense in VEP predictions / HIGH impact'
#         dbnsfp_annotation_complete_chr_final.loc[(dbnsfp_annotation_complete_chr_final['IMPACT'].str.contains('LOW|MODIFIER')) & (dbnsfp_annotation_complete_chr_final['score'].isna() == True) & (dbnsfp_annotation_complete_chr_final['Suppl_comments'].isna() == True), 'Suppl_comments'] = 'No missense in VEP predictions / LOW|MODIFIER impact'
#         dbnsfp_annotation_complete_chr_final['Suppl_comments'] = dbnsfp_annotation_complete_chr_final['Suppl_comments'].fillna('')
#         dbnsfp_annotation_complete_chr_final['comments'] = dbnsfp_annotation_complete_chr_final['comments'].astype(str) + ' - ' + dbnsfp_annotation_complete_chr_final['Suppl_comments'].astype(str)
        dbnsfp_annotation_complete_chr_final_bak2 = dbnsfp_annotation_complete_chr_final.copy()
#         print(dbnsfp_annotation_complete_chr_final_bak2)
        
#         print(dbnsfp_annotation_complete_chr_final.columns)


        # COMPUTE TEXT PREDICTIONS
        dbnsfp_annotation_complete_chr_final.loc[dbnsfp_annotation_complete_chr_final['score'] >= 0.5, 'pred'] = 'D'
        dbnsfp_annotation_complete_chr_final.loc[dbnsfp_annotation_complete_chr_final['score'] < 0.5, 'pred'] = 'T'
        dbnsfp_annotation_complete_chr_final.loc[dbnsfp_annotation_complete_chr_final['score'].isna() == True, 'pred'] = 'U'


#         dbnsfp_annotation_complete_chr_final.loc[(dbnsfp_annotation_complete_chr_final['IMPACT'].str.contains('HIGH')) & (dbnsfp_annotation_complete_chr_final['score'].isna() == True) , 'score'] = 1
#         dbnsfp_annotation_complete_chr_final.loc[(dbnsfp_annotation_complete_chr_final['IMPACT'].str.contains('LOW|MODIFIER')) & (dbnsfp_annotation_complete_chr_final['score'].isna() == True) , 'score'] = 0
        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), " - FINAL CLEAN : ", dbnsfp_annotation_complete_chr_final.shape))
        log.close()

        dbnsfp_annotation_complete_chr_final['sd'] = 0
        dbnsfp_annotation_complete_chr_final['comments'] = dbnsfp_annotation_complete_chr_final['comments'].fillna('*')
        
        
        template = pd.read_csv("/gstock/MISTIC/CAGI6/download/templates/dbNSFP4_nsSNV.chr{}.template.gz".format(chromosome), compression='gzip', sep='\t')
        template['ID'] = template['#chr'].astype(str) + "-" + template['pos(1-based)'].astype(str) + "-" + template['ref'].astype(str) + "-" + template['alt'].astype(str)

        dbnsfp_annotation_complete_chr_final = dbnsfp_annotation_complete_chr_final.dropna(subset=['aaref', 'aaalt'])
        dbnsfp_annotation_complete_chr_final['ID'] = dbnsfp_annotation_complete_chr_final['#chr'].astype(str) + "-" + dbnsfp_annotation_complete_chr_final['pos(1-based)'].astype(str) + "-" + dbnsfp_annotation_complete_chr_final['ref'].astype(str) + "-" + dbnsfp_annotation_complete_chr_final['alt'].astype(str)
        dbnsfp_annotation_complete_chr_final = dbnsfp_annotation_complete_chr_final.loc[dbnsfp_annotation_complete_chr_final['ID'].isin(template['ID'].values.tolist())]
        dbnsfp_annotation_complete_chr_final = dbnsfp_annotation_complete_chr_final[['#chr', 'pos(1-based)','ref','alt','score','sd','pred', 'comments']].drop_duplicates().sort_values(by=['#chr', 'pos(1-based)', 'ref', 'alt', 'comments'])
        
        dbnsfp_annotation_complete_chr_final[['#chr', 'pos(1-based)','ref','alt','score','sd','pred', 'comments']].to_csv(output_file_path, sep='\t', index=False)
        log = open('/gstock/MISTIC/CAGI6/download/OUTPUT/log.txt', 'a')
        log.write("{}{}\n".format(time.strftime("%m/%d/%Y, %H:%M:%S", time.localtime()), ' - FINAL EXPORTED'))
        log.close()
        
    #     test[['#chr', 'pos(1-based)','ref','alt','score','sd','pred', 'comments', 'IMPACT', 'Consequence']]

#         subprocess_args = [
#             "python",
#             "/gstock/MISTIC/CAGI6/download/validation.py",
#             output_file_path,
#         ]
#         p1 = subprocess.Popen(subprocess_args, stdout=subprocess.PIPE,)
#         p1.wait()
    else:
        print('File exists for chr {}'.format(chromosome))

In [92]:
# EXPORT TO SYNAPSE

import synapseclient
from synapseclient import File
syn = synapseclient.login("weber8thomas", "alpiance")

# Add a local file to an existing project (syn12345) on Synapse

dir_path = "/gstock/MISTIC/CAGI6/download/OUTPUT/"
list_dir = sorted([e for e in os.listdir(dir_path) if e.endswith('.tsv')])
for file in list_dir:
    print(file)
    file = File(path= dir_path + file, parent='syn26264707')

    file = syn.store(file)



Welcome, weber8thomas!

MISTIC_chr1.tsv

##################################################
 Uploading file to Synapse storage 
##################################################

MISTIC_chr10.tsv

##################################################
 Uploading file to Synapse storage 
##################################################

MISTIC_chr11.tsv

##################################################
 Uploading file to Synapse storage 
##################################################

MISTIC_chr12.tsv

##################################################
 Uploading file to Synapse storage 
##################################################

MISTIC_chr13.tsv

##################################################
 Uploading file to Synapse storage 
##################################################

MISTIC_chr14.tsv

##################################################
 Uploading file to Synapse storage 
##################################################

MISTIC_chr15.tsv

#################